In [13]:
%reset -f

In [14]:
import pandas as pd
import numpy as np
import json
import io
import os
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

from pandas import json_normalize

from bs4 import BeautifulSoup
import gspread as gs

from datetime import datetime
from google.cloud import bigquery

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [15]:
import MyFunc
# from MyFunc import cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *
# cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [16]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery


credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

In [17]:
query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'outcrapper_google_results_raw'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1


In [ ]:
query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.daftar_kota_admin_level_kelurahan_with_lat_long`

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data_location = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

In [18]:
%%time
import os
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

scrappe_date='20231207'
        
# data_location=pd.read_csv('daftar_kota_admin_level_kelurahan_with_lat_long.csv',sep=';')
# data_location=data_location[(data_location.latitude<=6) &(data_location.latitude>=-11)&(data_location.longitude>=95)&(data_location.longitude<=141)]
# data_location=data_location[data_location.Province.str.contains('Jakarta|Jawa|Banten|Yogyakarta')]

# df_temp=pd.read_csv('results/clear_raw_table.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     if column in ['working_hours','about']:
#         my_schema.append(bigquery.SchemaField(column, "JSON"))
#     else:
#         my_schema.append(bigquery.SchemaField(column, "STRING"))

path = 'outcrapper_raw/merchant_list/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.place_id!='__NO_PLACE_FOUND__']
        
        data['outcrapper_job_id']=file.split('_')[0]
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_data_merchants(data)
        
        # data=filtering_data_merchants(data,'Cashbac-GCP-Keys.json')
        
        data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        
        my_schema=[]
        save_columns=[]
        for column in df_temp.column_name :
            if column in data.columns :
                save_columns.append(column)
                if column in ['working_hours','about']:
                    my_schema.append(bigquery.SchemaField(column, "JSON"))
                else:
                    my_schema.append(bigquery.SchemaField(column, "STRING"))
        data[save_columns].to_csv('results/temp_upload_raw.csv',sep=';',index=False)
        
        #############################################################################################
        #BQ DATALAKE HOREGO
        # src_path_file='results/temp_upload_raw.csv'
        # target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        # bucket_name='horego-bq'
        # upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        # table_schema = "horego_datalake_dev"
        # table_name = "outcrapper_google_results_raw"
        # gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        # create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        
        #############################################################################################
        #############################################################################################
        #BQ DATALAKE CASHBAC
        bucket_name='cashbac_datalake'
        src_path_file='results/temp_upload_raw.csv'
        target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        table_schema = "cashbac_datalake_prod"
        table_name = "outcrapper_google_results_raw"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        #############################################################################################
        
        
path = 'outcrapper_raw/popular_brands/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.place_id!='__NO_PLACE_FOUND__']
        
        data['outcrapper_job_id']=file.split('_')[0]
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_data_merchants(data)
        
        # data=filtering_data_merchants(data,'Cashbac-GCP-Keys.json')
        
        data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        my_schema=[]
        save_columns=[]
        for column in df_temp.column_name :
            if column in data.columns :
                save_columns.append(column)
                if column in ['working_hours','about']:
                    my_schema.append(bigquery.SchemaField(column, "JSON"))
                else:
                    my_schema.append(bigquery.SchemaField(column, "STRING"))
        data[save_columns].to_csv('results/temp_upload_raw.csv',sep=';',index=False)
        
        #############################################################################################
        #BQ DATALAKE HOREGO
        # src_path_file='results/temp_upload_raw.csv'
        # target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        # bucket_name='horego-bq'
        # upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        # table_schema = "horego_datalake_dev"
        # table_name = "outcrapper_google_results_raw"
        # gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        # create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        
        #############################################################################################
        #############################################################################################
        #BQ DATALAKE CASHBAC
        bucket_name='cashbac_datalake'
        src_path_file='results/temp_upload_raw.csv'
        target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        table_schema = "cashbac_datalake_prod"
        table_name = "outcrapper_google_results_raw"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        #############################################################################################


path = 'outcrapper_raw/merchant_list_no_filter/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.place_id!='__NO_PLACE_FOUND__']
        
        data['outcrapper_job_id']=file.split('_')[0]
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_data_merchants(data)
        
        # data=filtering_data_merchants(data,'Cashbac-GCP-Keys.json')
        
        data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        
        my_schema=[]
        save_columns=[]
        for column in df_temp.column_name :
            if column in data.columns :
                save_columns.append(column)
                if column in ['working_hours','about']:
                    my_schema.append(bigquery.SchemaField(column, "JSON"))
                else:
                    my_schema.append(bigquery.SchemaField(column, "STRING"))
        data[save_columns].to_csv('results/temp_upload_raw.csv',sep=';',index=False)
        
        #############################################################################################
        #BQ DATALAKE HOREGO
        # src_path_file='results/temp_upload_raw.csv'
        # target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        # bucket_name='horego-bq'
        # upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        # table_schema = "horego_datalake_dev"
        # table_name = "outcrapper_google_results_raw"
        # gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        # create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        
        #############################################################################################
        #############################################################################################
        #BQ DATALAKE CASHBAC
        bucket_name='cashbac_datalake'
        src_path_file='results/temp_upload_raw.csv'
        target_path_file=f"scrapping_results/raw/{file.replace('.json','.csv')}"
        upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        table_schema = "cashbac_datalake_prod"
        table_name = "outcrapper_google_results_raw"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/raw/{file.replace('.json','.csv')}"
        create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        #############################################################################################

del data        
print('DONE')

202312071014266b2e_places_outlet_submission_7_des.json


Uploading results/temp_upload_raw.csv
Path in GCS: cashbac_datalake/scrapping_results/raw/202312071014266b2e_places_outlet_submission_7_des.csv
table on : cashbac_datalake_prod.outcrapper_google_results_raw
DONE
CPU times: user 255 ms, sys: 63.1 ms, total: 318 ms
Wall time: 4.04 s
